In [21]:
import pandas as pd
import numpy as np
import datetime
from collections import OrderedDict
import seaborn as sns

fundamentals = init_fundamentals()

today = datetime.datetime.now()
info_date = datetime.date(today.year, today.month, today.day)

In [22]:
stocks_df = get_fundamentals(query(fundamentals.company_reference.sid,
                                    fundamentals.asset_classification.morningstar_sector_code,
                                    fundamentals.share_class_reference.symbol,
                                    fundamentals.company_reference.standard_name,
                                    fundamentals.valuation.market_cap,
                                    fundamentals.earnings_report.normalized_basic_eps,
                                    fundamentals.earnings_report.normalized_diluted_eps,
                                    fundamentals.valuation_ratios.pe_ratio,
                                    fundamentals.asset_classification.growth_grade,
                                    fundamentals.asset_classification.growth_score,
                                    fundamentals.valuation_ratios.sustainable_growth_rate,
                                    fundamentals.earnings_ratios.diluted_cont_eps_growth,
                                    fundamentals.earnings_ratios.diluted_eps_growth,
                                    fundamentals.earnings_ratios.dps_growth,
                                    fundamentals.earnings_ratios.equity_per_share_growth,
                                    fundamentals.operation_ratios.revenue_growth
                                  )
        # Picking one company as a proof of concept                    
        .filter(fundamentals.share_class_reference.symbol == 'AAPL' )                  
        ,info_date)

In [23]:
#transposing so stocks and info are in rows rather than columns
stocks_fund = stocks_df.transpose()

#getting all the stock symbols so i can ensure they are valid
stock_symbols = stocks_fund['symbol']

#creating array to put symbols in after they are validated
valid_symbols =[]

#validate symbols look, checking to see if i can get price, if i can't get the price, i skip it
for s in stock_symbols:  
    try:  
        data = get_pricing(
            s,
            fields ='close_price',
            start_date='2016-06-01', #customize your pricing date range
            end_date = '2016-06-01', 
            frequency='daily', #change to daily for daily pricing
        )
        valid_symbols.append(s)  
    except:  
        pass 


#get the prices for the valid stock symbols, no error should be thrown
data = get_pricing(
    valid_symbols,
    fields ='close_price',
    start_date='2016-06-01', #customize your pricing date range
    end_date = '2016-06-01', 
    frequency='daily', #change to daily for daily pricing
)

#tranpose so stocks are in rows
data = data.transpose()

#rename 1st column
data.columns = ['current_price']

#join tables on the stock index
all_stock_info = pd.concat([data, stocks_fund], axis=1, join_axes=[data.index])

In [24]:
#sanity check
all_stock_info.head()

,current_price,sid,morningstar_sector_code,symbol,standard_name,market_cap,normalized_basic_eps,normalized_diluted_eps,pe_ratio,growth_grade,growth_score,sustainable_growth_rate,diluted_cont_eps_growth,diluted_eps_growth,dps_growth,equity_per_share_growth,revenue_growth
Equity(24 [AAPL]),98.45,Asset(24 [AAPL]),311,AAPL,Apple Inc,5.34323e+11,1.907014,1.897891,10.8712,A,77.93729,0.3001,-0.18388,-0.184549,0.106383,-0.049858,-0.128478


In [39]:
#constants in Graham's formula
R = 4.4
y = 3.11

#doing the calculations for calcualted price, growth, calculated growth

all_stock_info['cal_price'] = (all_stock_info['normalized_basic_eps']*(all_stock_info['pe_ratio']+\
                                               (1.5*((all_stock_info['sustainable_growth_rate']+\
                                                      all_stock_info['revenue_growth']))*100))*R)/y

all_stock_info['growth'] = (((all_stock_info['current_price']*y)/\
          (R*all_stock_info['normalized_basic_eps']))- all_stock_info['pe_ratio'])/1.5

all_stock_info['cal_growth'] = 100* (all_stock_info['sustainable_growth_rate']\
                                     + all_stock_info['revenue_growth'])


In [41]:
columns = ['stock_ticker', 'cur_price', 'cal_price', 'cur_growth', 'cal_growth']
grahams_vs_actuals = pd.DataFrame(columns=columns)
grahams_vs_actuals['stock_ticker'] = all_stock_info['symbol']
grahams_vs_actuals['cur_price'] = all_stock_info['current_price']
grahams_vs_actuals['cal_price'] = all_stock_info['cal_price']
grahams_vs_actuals['cur_growth'] = all_stock_info['growth']
grahams_vs_actuals['cal_growth'] = all_stock_info['cal_growth']

